# Landsat Collection 2 downloader
abolfazl.irani@eawag.ch

The code is originally from Michael Brechbühler (eawag).

This notebook uses the [landsatxplore](https://github.com/yannforget/landsatxplore) python package to download Landsat Collection 2 scenes directly from USGS.

### Setup environment
To setup a new environment (using conda):
> conda create --name landsatxplore -c conda-forge python=3 notebook nb_conda_kernels geopandas hvplot cartopy geoviews

To setup a new environment (using conda and environment.yml):
> conda env create -n landsatxplore --file environment.yml

### Imports

In [7]:
import geopandas as gpd
import pandas as pd
import os
import matplotlib.pyplot as plt
import shapely
import hvplot.pandas
from tqdm import tqdm
from pathlib import Path
import tarfile
from contextlib import closing

import landsatxplore
from landsatxplore.earthexplorer import EarthExplorer
from landsatxplore.api import API

os.chdir("../..")

## 1. Search query

### USGS EROS credentials
To allow queries and file downloads from the USGS EROS webserver a free account is necessary (https://ers.cr.usgs.gov/register)

In [2]:
# Your USGS  credentials

username = "nls_rtz_1"
password = "b2XRiOz5Cmt9uYIfwtFq"

Now we can set the desired search parameters for our Landsat query. We also have to define which Landsat collection to use.

| Dataset Name | Dataset ID |
| :-- | -- |
| Landsat 5 TM Collection 2 Level 1 | `landsat_tm_c2_l1` |
| Landsat 5 TM Collection 2 Level 2 | `landsat_tm_c2_l2` |
| Landsat 7 ETM+ Collection 2 Level 1 | `landsat_etm_c2_l1` |
| Landsat 7 ETM+ Collection 2 Level 2 | `landsat_etm_c2_l2` |
| Landsat 8 Collection 2 Level 1 | `landsat_ot_c2_l1` |
| Landsat 8 Collection 2 Level 2 | `landsat_ot_c2_l2` |
| Landsat 9 Collection 2 Level 1 | `landsat_ot_c2_l1` |
| Landsat 9 Collection 2 Level 2 | `landsat_ot_c2_l2` |

### Search parameters

In [23]:
# Search parameters
# Load AOIs
PATH_VIIRS_PERIMETERS = "data/feature_layers/fire_atlas/"

# Load VIIRS perimeters in Siberian tundra
FN_VIIRS_CAVM_PERIMETERS = os.path.join(PATH_VIIRS_PERIMETERS,
                                        "viirs_perimeters_in_cavm_e113.gpkg")

if os.path.exists(FN_VIIRS_CAVM_PERIMETERS):
    print("CAVM Fire perimeter file exists, loading.")
    merged_fire_perimeters = gpd.read_file(FN_VIIRS_CAVM_PERIMETERS)
else:
    print("Please prepare and filter the VIIRS fire perimeters using\n \"0_preprocess_ancillary_data.py\" ")
    pass

TEST_ID = 14211 # fire ID for part of the large fire scar

perimeter = merged_fire_perimeters.loc[merged_fire_perimeters.fireid==TEST_ID]

# Get rectangular bbox of selected fire perimeter (lonmin, latmin, lonmax, latmax)
bbox = tuple(list(perimeter.total_bounds)) 

# List of months to search data
months = [5,6,7,8,9,10]

search_params = {
    'dataset': 'landsat_ot_c2_l2',
    'bbox': bbox, 
    'start_date': f'{perimeter.tst_year.item()-1}-05-01',
    'end_date': f'{perimeter.tst_year.item()}-10-31',
    'months': months,
    'max_cloud_cover': 100,
    'max_results': 1000, # Defaults to 100 if not specified
}

CAVM Fire perimeter file exists, loading.


### Plot ROI

In [9]:
# Plot bounding box on map
bbox_poly = shapely.geometry.box(*bbox, ccw=True)
df_bbox = gpd.GeoDataFrame([], geometry=[bbox_poly])

plot_bbox = df_bbox.hvplot(title='ROI bounding box', geo=True, 
                           fill_alpha=0.2, color='red', line_color='red', 
                           #tiles='CartoLight'
                           tiles='OSM', label='ROI'
                          )

plot_bbox

:Overlay
   .WMTS.I       :WMTS   [Longitude,Latitude]
   .Polygons.ROI :Polygons   [Longitude,Latitude]

### Start search query

In [24]:
# initialize a new API instance
api = API(username, password)

# search for Landsat 8-9 L2C2 scenes
scenes = api.search(**search_params)

# logout
api.logout()

# create a GeoDataFrame from the returned metadata
df_scenes = pd.DataFrame(scenes)
df_scenes = df_scenes[['display_id','wrs_path', 'wrs_row',
                       'satellite','cloud_cover','acquisition_date', 'spatial_coverage']]
df_scenes.sort_values('acquisition_date', 
                      ascending=False, inplace=True)
df_scenes['satellite'] = df_scenes.satellite.apply(lambda x: f'Landsat-{x}')
df_scenes['tile'] = df_scenes.display_id.apply(lambda x: x.split('_')[2][:3]+'/'+x.split('_')[2][3:])
gdf_scenes = gpd.GeoDataFrame(df_scenes.drop(columns='spatial_coverage'), 
                              geometry=df_scenes.spatial_coverage,
                              crs='EPSG:4326')

gdf_scenes.head()

,display_id,wrs_path,wrs_row,satellite,cloud_cover,acquisition_date,tile,geometry
0,LC08_L2SP_116010_20201005_20201016_02_T1,116,10,Landsat-8,72,2020-10-05,116/010,"POLYGON ((143.74545 70.49464, 148.31032 69.736..."
1,LC08_L2SP_118010_20201003_20201015_02_T1,118,10,Landsat-8,46,2020-10-03,118/010,"POLYGON ((140.658 70.49465, 145.22279 69.73638..."
2,LC08_L2SP_117010_20200926_20201006_02_T1,117,10,Landsat-8,0,2020-09-26,117/010,"POLYGON ((142.20498 70.49475, 146.7698 69.7364..."
3,LC08_L2SP_119009_20200924_20201006_02_T1,119,9,Landsat-8,55,2020-09-24,119/009,"POLYGON ((140.98635 71.8195, 145.78747 71.0154..."
4,LC08_L2SP_119010_20200924_20201006_02_T1,119,10,Landsat-8,4,2020-09-24,119/010,"POLYGON ((139.11441 70.49452, 143.67915 69.736..."


### Plot available tiles over ROI

In [25]:
gdf_tiles = gdf_scenes.groupby('tile').first().reset_index()[['tile', 'wrs_path', 'wrs_row', 'geometry']].set_crs('EPSG:4326')
gdf_labels = gdf_tiles.set_geometry(gdf_tiles.centroid)

plot_labels = gdf_labels.to_crs("EPSG:3857").assign(x=lambda df: df.geometry.x, y=lambda df: df.geometry.y).hvplot.labels(text="tile", x="x", y="y", text_color='white')

plot_tiles = gdf_tiles.hvplot(title='Available tiles over ROI', geo=True,
                              fill_alpha=0.3, line_color='blue',
                              height=800, tiles='OSM', label='Landsat tile'
                             ) * plot_bbox * plot_labels

plot_tiles

C:\Users\nrietze\AppData\Local\Temp\ipykernel_12228\3331222002.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_labels = gdf_tiles.set_geometry(gdf_tiles.centroid)


:Overlay
   .WMTS.I                :WMTS   [Longitude,Latitude]
   .Polygons.Landsat_tile :Polygons   [Longitude,Latitude]
   .WMTS.II               :WMTS   [Longitude,Latitude]
   .Polygons.ROI          :Polygons   [Longitude,Latitude]
   .Labels.I              :Labels   [x,y]   (tile)

## 2. Download scenes
### Filter and select scene IDs

In [30]:
# Select scene ids to download from dataframe
#ids = df_scenes.display_id.values # all
# df_scenes_filt = df_scenes.loc[(df_scenes.cloud_cover < 2) & (df_scenes.tile=='196/028')] # filter by cloudcover and tile
# filter by cloudcover and tile
df_scenes_filt = df_scenes.loc[(df_scenes.cloud_cover <= 80)] 
print(df_scenes_filt.shape)
print(df_scenes_filt.head())

(66, 8)
                                 display_id  wrs_path  wrs_row  satellite  \
0  LC08_L2SP_116010_20201005_20201016_02_T1       116       10  Landsat-8   
1  LC08_L2SP_118010_20201003_20201015_02_T1       118       10  Landsat-8   
2  LC08_L2SP_117010_20200926_20201006_02_T1       117       10  Landsat-8   
3  LC08_L2SP_119009_20200924_20201006_02_T1       119        9  Landsat-8   
4  LC08_L2SP_119010_20200924_20201006_02_T1       119       10  Landsat-8   

   cloud_cover acquisition_date  \
0           72       2020-10-05   
1           46       2020-10-03   
2            0       2020-09-26   
3           55       2020-09-24   
4            4       2020-09-24   

                                    spatial_coverage     tile  
0  POLYGON ((143.74545 70.49464, 148.31032 69.736...  116/010  
1  POLYGON ((140.658 70.49465, 145.22279 69.73638...  118/010  
2  POLYGON ((142.20498 70.49475, 146.7698 69.7364...  117/010  
3  POLYGON ((140.98635 71.8195, 145.78747 71.0154...  119/009 

### Download .tar files

In [15]:
# set output directory
path_output_dir = Path('data/C2L2')

# initialize the API
ee = EarthExplorer(username, password)

# Get scene IDs for download
ids = df_scenes_filt.display_id.values[:5] #just 5 to test

# download the scenes
for id in tqdm(ids, total=len(ids), desc="Total download progress", position=-1):
    path_output_file = path_output_dir.joinpath(id+'.tar')
    try:
        ee.download(id, output_dir=path_output_dir)
        print('{} successful'.format(id))

  # aditional error handling
    except:
        if path_output_file.exists():
            print(f'{id} error but file exists')
        else:
            print(f'{id} error')

ee.logout()

Download failed with dataset id 1 of 3. Re-trying with the next one.


895MB [01:13, 12.7MB/s]                            


None of the archived ids succeeded! Update necessary!
LC09_L2SP_116010_20231006_20231007_02_T1 error but file exists
Download failed with dataset id 1 of 3. Re-trying with the next one.


100%|██████████| 907M/907M [00:36<00:00, 26.4MB/s] 


None of the archived ids succeeded! Update necessary!
LC08_L2SP_117010_20231005_20231011_02_T1 error but file exists
Download failed with dataset id 1 of 3. Re-trying with the next one.


...

None of the archived ids succeeded! Update necessary!
LC09_L2SP_118010_20231004_20231005_02_T1 error but file exists
Download failed with dataset id 1 of 3. Re-trying with the next one.


900MB [01:01, 15.3MB/s]                             


None of the archived ids succeeded! Update necessary!
LC08_L2SP_119010_20231003_20231011_02_T1 error but file exists
Download failed with dataset id 1 of 3. Re-trying with the next one.


875MB [03:24, 4.48MB/s]                            
Total download progress: 100%|██████████| 5/5 [11:21<00:00, 136.29s/it]

None of the archived ids succeeded! Update necessary!
LC08_L2SP_116010_20230928_20231003_02_T1 error but file exists


In [59]:
df_scenes_filt.to_csv(path_output_dir.joinpath('metadata_L8L9_'+search_params['start_date']+'to'+search_params['end_date']+'.csv'))

### Extract .tar files

In [16]:
path_output_dir

WindowsPath('data/C2L2')

In [17]:
# Extract .tar files into folders
paths_output = list(path_output_dir.glob('*.tar'))

for path in tqdm(paths_output, desc="Total extraction progress"):
    with closing(tarfile.open(path)) as fl:
        path_output_folder = path_output_dir.joinpath(path.name.split('.')[0])
        path_output_folder.mkdir(parents=True, exist_ok=True)
        fl.extractall(path_output_folder)
    path.unlink()

Total extraction progress: 100%|██████████| 5/5 [00:07<00:00,  1.54s/it]
